In [0]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Get:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:12 https://developer.download.nvidia.com/compute/cuda

In [0]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [0]:
# 크롬 드라이버에 옵션 주기
options = webdriver.ChromeOptions()
options.add_argument('--headless') # 인터페이스 없는
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--window-size=1920x1080')

In [0]:
complaint = pd.DataFrame(columns=['Subject', 'Content'])

# driver 변수를 Chrome webdriver 클래스로 만든다. 
listPage = webdriver.Chrome('chromedriver', options=options)

# 3초간 기다리자.
listPage.implicitly_wait(3)

# 페이지에서 소스 가져오기
listPage.get('https://eungdapso.seoul.go.kr/Exp/Exp01/Exp01_lis.jsp')

# 5초간 기다려서 javascript 내용이 뜰때까지 기다리기
time.sleep(5)

startDate = listPage.find_element_by_id("StartDate")
startDate.send_keys(webdriver.common.keys.Keys.CONTROL, 'a')
startDate.send_keys("2010-10-10")

searchButton = listPage.find_element_by_xpath('//*[@id="content_cont"]/div[2]/div/form/table/tbody/tr[2]/td/div/a')
listPage.execute_script("arguments[0].click();", searchButton)

In [0]:
# 마지막 페이지 번호 추출
pageNumStr = listPage.find_element_by_xpath('//*[@id="content_cont"]/div[2]/div/div/div[11]/a[2]').get_attribute("href").split("'")[1]
pageNumInt = int(pageNumStr)+1

175

In [0]:
for x in range(1,pageNumInt):
  # 페이지 번호 클릭
  currentPageNum = listPage.find_element_by_xpath('//*[@id="content_cont"]/div[2]/div/div/div[11]/span['+str(x)+']')
  listPage.execute_script("arguments[0].click();", currentPageNum)

  # 민원 내용 크롤링
  for y in range(1, 11):
    # 민원 상세 정보로 이동
    postPage = listPage.find_element_by_xpath('//*[@id="content_cont"]/div[2]/div/div/div['+str(y)+']/ul/li[2]/a')
    listPage.execute_script("arguments[0].click();", postPage)

    # 민원 분야
    postSubject = listPage.find_element_by_xpath('//*[@id="content_cont"]/div[2]/div/table[1]/tbody/tr[1]/td').text
    postSubject = postSubject

    # 민원 내용
    postContent = listPage.find_element_by_xpath('//*[@id="content_cont"]/div[2]/div/table[2]/tbody/tr[1]/td/div').text
    # 민원 내용 중 필요없는 부분 제거
    postContent = postContent.replace('Q. 상담내용','')
    postContent = postContent.replace('\n','')

    # 민원 분야 및 민원 내용 DataFrame에 저장
    complaint = complaint.append(pd.DataFrame([[postSubject, postContent]], columns=['Subject', 'Content']), ignore_index=True)
    # 이전 페이지로 복귀
    listPage.back()

  # 다음 목록으로 가기
  if x%10 == 0:
    nextPageNum = listPage.find_element_by_xpath('//*[@id="content_cont"]/div[2]/div/div/div[11]/a[1]')
    listPage.execute_script("arguments[0].click();", nextPageNum)

In [0]:
complaint.tail()

,Subject,Content
0,환경/공원/상수도,Q. 상담내용\n강서구 OO동 OO길 OO 옆에 있는 헌옷 수거함을 다른 곳이 옮겨...
1,문화/관광/체육/디...,Q. 상담내용\n광화문 거주 주민입니다\n이런 페스티벌을 왜 광화문에서 하나요?\n...
2,환경/공원/상수도,Q. 상담내용\n종량제 50리터 쓰레기봉투의 크기가\n어떤 쓰레기통 기준으로 만들어...
3,교통,Q. 상담내용\n서울대병원 앞에서 차없는거리 좀 안하면 좋겠습니다. 한시가 급한 응...
4,교통,Q. 상담내용\n남부터미널역안 화장실이 문짝등.\n많이 낡고 고장난칸도 있고\n불편합니다


In [0]:
complaint.to_csv("Complaint.csv", encoding='utf-8', index=False)